<a href="https://colab.research.google.com/github/wevbarker/PSALTer/blob/main/advection_equation_problem_set.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Before we start:

## Setup

In [ ]:
%matplotlib inline
%matplotlib notebook
import numpy as np
from IPython.display import HTML
import matplotlib.pyplot as plt
import matplotlib.animation as animation

**Important note**: You might need to install ipyml. To do so, use either <code>pip install ipympl</code> or <code>conda install -c conda-forge ipympl</code>, depending on your Python distribution. You might also have to replace <code>%matplotlib notebook</code> by <code>%matplotlib ipyml</code> in the previous cell (and re-run it)

## Hints

$ \bullet $ Don't forget the periodic boundary conditions when writing your schemes. \
$ \bullet $ The method <code>np.roll</code> will help you a lot. But if you want a small challenge, you can decide not to use it. \
$ \bullet $ In Python, <code> f[:] = </code> updates the content of the array whereas <code> f = </code> creates a new array. \
$ \bullet $ You can pause an animation called "anim" using <code>anim.pause()</code>, and resume it using <code>anim.resume()</code>. \
$ \bullet $ There are several ways to define the Fourier transform. They all work, but be careful of using the same convention when taking the Fourier transform and the inverse Fourier transform. \
$ \bullet $ When writing your spectral scheme, don't write your own Fourier transform. Use the functions provided by numpy, <code> np.fft.fft </code> and <code> np.fft.ifft </code>. \
$ \bullet $ Numerical implementations of FFT commonly rearrange the positive and negative frequencies. For A = fft(a), the zero frequency component is ```A[0]```, the positive frequency components are ```A[1:N/2]```, and the negative frequencies are ```[N/2+1:]```. Numpy's ```fftfreq``` function returns frequencies $k$ for a given grid spacing $\Delta x$ and takes this reordering into account. \
$ \bullet $ When writing your spectral code, you still have to solve an ODE. Remember the lecture on ODEs and make sure to choose a scheme that is stable.

# Solving partial differential equations

## Goal

Write three programs that all calculate the solution to the same advection equation $ \frac{\partial f}{\partial t} = - v \frac{\partial f}{\partial x} $.

Compare your programs to the analytical results, and to each other. Conclude on the strength and weaknesses of each approach.

## Physics

Partial differential equations (PDEs) are omnipresent in physics. They are our best tools to represent the behavior of fluids (Navier-Stokes equations), of quantum-mechanical systems (Schrödinger equation), of spacetime (Einstein field equation), of electromagnetic waves (Maxwell's equations), ...

PDEs also have a lot of industrial applications. In aerospace engineering, they help optimize the shape of plane wings. In architecture, they help ensure the structural integrity of buildings. In micro-electronics, they help reduce the leakage currents of chips. This led to the development of many professional, millions-of-line-long codes that specialise in solving PDEs. Examples include Abaqus, ANSYS, ...

Fortunately, physicists are generally faced with simpler PDEs than engineers, at least in terms of geometry. This problem offers to face the cleanest of PDEs, the advection equation, and to solve it using three different 10-lines-long codes. This is enough to introduce the concepts of convergence and stability, which are the first things to consider when choosing a PDE solver.

The advection equation is the equation that governs the evolution of a scalar field ${ f }$ that is advected at a constant velocity ${ v }$. For instance, you could use the advection equation to predict the future position and shape of an ink drop that you just put in a river. The advection equation writes

$$
\begin{cases}
    f (x, t = 0) = f_{0} (x) , \\
    \frac{\partial f}{\partial t} = - v \frac{\partial f}{\partial x} .
\end{cases}
$$

The first equation is the initial condition (how much ink did you put, and where?), and the second equation is the evolution equation (how does the ink move?). The analytical solution is quite simply

$$
f(x, t) = f_{0} (x - v t).
$$
This analytical solution can be obtained using the [method of characteristics](https://en.wikipedia.org/wiki/Method_of_characteristics#Example).

For this problem, we will set ${ f_{0} = \mathrm{e}^{ - x^{2} / 2 } }$ and ${ v = 0.1 }$. We will also restrict the spatial domain to ${ x \in [-5, 5] }$ and assume that ${ x = -5 }$ and ${ x = 5 }$ represent the same point. This is called using periodic boundary conditions. Finally, we will integrate the equation for a time ${ t_{\text{max}} = 100 }$.

This way, the analytical solution at time ${ t_{\text{max}} }$ is ${ f_{0} }$. This will allow us to check our algorithms easily.

In [ ]:
# Let's define the parameters

# physics
v = 10.0**(-1) # advection speed

# domain
x_min = -5.0
x_max = 5.0

# initial condition
class Gaussian:
    def __init__(self, center = 0.0, width = 1.0, amplitude = 1.0):
        self.center = center
        self.width = width
        self.amplitude = amplitude

    def __call__(self, x):
        return self.amplitude * np.exp(- (x - self.center)**2 / (2 * self.width**2))

gaussian = Gaussian()
    # gaussian is a function that can take a scalar and return a scalar,
    # or take a numpy array and return a numpy array

In [ ]:
# let's see what the initial condition looks like

X_example = np.linspace(x_min, x_max, 10**2, endpoint = True)
f0_example = gaussian(X_example)

fig = plt.figure()
ax = fig.gca()

ax.set_xlim(x_min, x_max)
ax.set_ylim(-0.1, 1.5)

ax.set_xlabel(r'$x$')
ax.set_ylabel(r'$f_{0}(x)$')

plt.plot(X_example, f0_example, 'b--')

plt.show()

<IPython.core.display.Javascript object>

## Tasks

### Core task 1

Our first algorithm will use the explicit upwind scheme. It is arguably the simplest algorithm within the class of finite difference schemes.

The first idea is to discretize time and space. We will have instants ${ 0, \Delta t, 2 \Delta t, ..., T }$ and positions ${ -5, -5 + \Delta x, -5 + 2 \Delta x, ..., 5 - \Delta x }$. It is common practice to denote the time-frames with the subscript ${ ^{n} }$ and the cells with the superscript ${ _{i} }$. This leads to the notation

$$
f(n \Delta t, -5 + i \Delta x) = f(t^{n}, x_{i}) = f^{n}_{i} .
$$

In [ ]:
# Let's define the parameters

# grid
n_points_space = 10**3
dx = (x_max - x_min) / n_points_space
dt = 0.05

The second idea is to replace the time derivative on the left hand side by
$$
\frac{f^{n+1}_{i} - f^{n}_{i}}{\Delta t} ,
$$
and the space derivative on the right-hand side by
$$
\frac{f^{n}_{i} - f^{n}_{i - 1}}{\Delta x} .
$$
From the definition of the partial derivative, it is intuitive that if ${ \Delta t }$ and ${ \Delta x }$ are both small, this replacement should not cause a big error.

This trick leaves us with the equation
$$
f^{n+1}_{i} = f^{n}_{i} + dt \times -v \frac{f^{n}_{i} - f^{n}_{i - 1}}{\Delta x}
$$
which gives ${ f^{n+1} }$ from ${ f^{n} }$. This will allow us to easily the initial condition time-frame by time-frame all the way to ${ t_{\text{max}} }$.

**Implement the explicit upwind scheme by completing the following piece of code.**

In [ ]:
def Upwind_explicit(v, n_points_space, dx, dt):
    df_dt = np.zeros(n_points_space, dtype = np.float64)

    def step(f):
        df_dt[0] = -v * (f[0] - f[-1]) / dx
        df_dt[1:] = -v * (f[1:] - f[:-1]) / dx

        f[:] += dt * df_dt

    return step

**To sanity-check on your code, run the following piece of code and convince yourself that your scheme solves the advection equation correctly**

In [ ]:
# initial condition
X = np.linspace(x_min, x_max, n_points_space, endpoint = True)
f0 = gaussian(X)
f = f0.copy()

# scheme
step = Upwind_explicit(v, n_points_space, dx, dt)

# preparation of the animation
fig = plt.figure()
ax = fig.gca()

ax.set_xlim(x_min, x_max)
ax.set_ylim(-0.1, 1.5)

ax.set_xlabel(r'$x$')
ax.set_ylabel(r'$f(x, t)$')

show_f0, = ax.plot(X, f0, 'b--', label = "f0")
show_ft, = ax.plot([], [], 'b', lw = 1, ms = 2, label = "f")
time = ax.annotate(0, xy = (3, 1.4), xytext = (3, 1.4))

# integration
def animate(i):
    step(f)

    show_ft.set_data(X, f)

    time.set_text("t / T = %.2f" % (v * i * dt / (x_max - x_min)))

    return show_f0, show_ft, time

# creation of the animation
anim = animation.FuncAnimation(fig, animate, 10**5, interval = dt * 10, blit = True)

# show the results
plt.show()

<IPython.core.display.Javascript object>

**Explore the errors made by your code. What happens if you wait for a few periods?**

When we run the explicit upwind scheme for several periods, we start to see two main errors:

1.   **Numerical Dissipation**:
The scheme adds artificial diffusion, which causes the amplitude of the wave (like a Gaussian bump) to decrease over time. Essentially, the sharp features of the initial condition get smoothed out. This happens because the first-order upwind method is quite diffusive, so as the wave circulates through the domain, its peak value decays and the profile broadens.
2.   **Phase Error**:
Besides losing amplitude, the wave doesn't travel at the exact physical speed. Instead, it usually lags behind the true position. This phase error accumulates over multiple periods, so the wave ends up being out of sync with what we would expect from the analytic solution.

Overall, after a few periods the numerical solution becomes both damped and shifted relative to the exact solution.



**Run the same code, but with a larger ${ \Delta t }$ of 1.1. What happens?**

If we increase Δt to 1.1, the explicit upwind scheme violates the CFL condition (which roughly requires that
∣𝑣Δ𝑡/Δ𝑥∣≤1). When this condition isn't met, the numerical method becomes unstable. In practice:

The solution may exhibit wild oscillations.
The values might blow up (i.e., become extremely large) or even turn into NaNs.
The scheme loses any physical meaning because the time step is too large to capture the correct propagation of the wave.
This instability occurs because the method cannot correctly propagate information when the time step is so big relative to the spatial grid spacing, leading to exponential error growth.

### Core task 2

Our second algorithm will use the Lax-Wendroff scheme, which is also a finite-difference method. It is slightly more complex, but as a reward it gives results that are closer to the truth.

The first idea is that the way we replaced the derivatives by finite differences was quite arbitrary. This time, we will use a different formula. Let us replace the spatial derivative by
$$
\frac{f^{n}_{i+1} - f^{n}_{i-1}}{2 \Delta x} .
$$

The second idea is to work around the expected numerical diffusion by adding a fictitious "retro-diffusion" term. Without going into the mathematical details, it is possible to predict how much diffusion a finite difference will cause, and to add just the right amount of "retro-diffusion". The final equation is
$$
\frac{f^{n+1}_{i} - f^{n}_{i}}{\Delta t} = -v \frac{f^{n}_{i+1} - f^{n}_{i-1}}{2 \Delta x} + \frac{v^{2} \Delta t}{2} \frac{f^{n}_{i+1} - 2 f^{n}_{i} + f^{n}_{i-1}}{\Delta x^{2}} .
$$

**Implement the Lax-Wendroff scheme.**

In [ ]:
def Lax_Wendroff(v, n_points_space, dx, dt):

    def step(f):
        f[:] = ( f - v * dt / (2 * dx) * (np.roll(f, -1) - np.roll(f, 1)) + (v**2 * dt**2) / (2 * dx**2) * (np.roll(f, -1) - 2 * f + np.roll(f, 1)) )

    return step

 **Sanity-check your code.**

In [ ]:
# initial condition
X = np.linspace(x_min, x_max, n_points_space, endpoint = True)
f0 = gaussian(X)
f = f0.copy()

# scheme
step = Lax_Wendroff(v, n_points_space, dx, dt)

# preparation of the animation
fig = plt.figure()
ax = fig.gca()

ax.set_xlim(x_min, x_max)
ax.set_ylim(-0.1, 1.5)

ax.set_xlabel(r'$x$')
ax.set_ylabel(r'$f(x, t)$')

show_f0, = ax.plot(X, f0, 'b--', label = "f0")
show_ft, = ax.plot([], [], 'b', lw = 1, ms = 2, label = "f")
time = ax.annotate(0, xy = (3, 1.4), xytext = (3, 1.4))

# integration
def animate(i):
    step(f)

    show_ft.set_data(X, f)

    time.set_text("t / T = %.2f" % (v * i * dt / (x_max - x_min)))

    return show_f0, show_ft, time

# creation of the animation
anim = animation.FuncAnimation(fig, animate, 10**5, interval = dt * 10, blit = True)

# show the results
plt.show()

<IPython.core.display.Javascript object>

**Run a few experiments to determine under which condition the Lax-Wendroff scheme is stable**

The experiments confirm that the Lax-Wendroff scheme is stable as long as the CFL condition
$$
\frac{v\,\Delta t}{\Delta x} \le 1
$$
is satisfied. When this condition is violated, the solution becomes unstable, exhibiting large oscillations and eventual blowup.


**Is the Lax-Wendroff scheme diffusive? What happens if you wait for several period?**

The Lax-Wendroff scheme is a second-order accurate finite-difference method for solving hyperbolic equations. Compared to first-order schemes like the explicit upwind method, the Lax-Wendroff scheme is designed to be *less diffusive*. This means that it introduces significantly less numerical diffusion (or artificial smoothing) into the solution.

However, while the scheme minimizes diffusion, it does not completely eliminate numerical errors. In fact, the Lax-Wendroff method is known to introduce *dispersive errors*. These errors manifest as oscillations or ripples near sharp gradients or discontinuities, and are particularly noticeable when the solution is not smooth.

1.   **Amplitude Preservation**: The main feature of the wave (e.g., the peak of a Gaussian) is much better preserved compared to more diffusive schemes like the upwind method. The overall amplitude does not decay as quickly.
2.   **Phase Errors**: Although the amplitude is maintained, slight phase errors may accumulate. This means that the wave may not travel at exactly the correct speed, causing a small shift in position over time.
3.   **Dispersive Oscillations**: Especially near steep gradients, you might observe small oscillatory ripples. These are a result of the dispersive nature of the Lax-Wendroff scheme, but they are generally less severe than the smearing effect of numerical diffusion.

In summary, the Lax-Wendroff scheme is not significantly diffusive compared to first-order methods, and over several periods it maintains the shape and amplitude of smooth initial conditions relatively well. However, the trade-off is the appearance of dispersive oscillations and slight phase errors.



**Use the code below to measure the error made by the Lax-Wendroff scheme as a function of ${ \Delta t }$ and ${ \Delta x }$. How does the error scale?**

In [ ]:
N = [(i + 1) * 10**2 for i in range(10)] + [(i + 1) * 10**3 for i in range(10)]
Errors = [0.0 for i in range(20)]

for i in range(20):
    n = N[i]

    # grid
    n_points_space = n
    dx = (x_max - x_min) / n_points_space
    n_points_time = 2 * n
    dt = 100 / n_points_time

    # initial condition
    X = np.linspace(x_min, x_max, n_points_space, endpoint = True)
    f0 = gaussian(X)
    f = f0.copy()

    # scheme
    step = Lax_Wendroff(v, n_points_space, dx, dt)

    # integration
    t = 0.0
    for j in range(n_points_time):
        step(f)
        t += dt

    # measure error
    error = np.max(np.abs(f - f0))
    Errors[i] = error

# plot the results
fig = plt.figure()
ax = fig.gca()

ax.set_xlim(N[0], N[-1])
ax.set_ylim(10**(-6), 10**0)

ax.set_xlabel(r'$n$')
ax.set_ylabel(r'$Error \,\,\, \epsilon$')

ax.set_xscale('log')
ax.set_yscale('log')

plt.plot(N, Errors, 'b')

plt.show()

<IPython.core.display.Javascript object>

The Lax-Wendroff scheme is a second-order method in both space and time. This implies that the local truncation error scales as
$$
O(\Delta x^2) \quad \text{and} \quad O(\Delta t^2).
$$
In our experiment, the time step is chosen as
$$
\Delta t = \frac{100}{2n} = \frac{50}{n},
$$
and the spatial step is
$$
\Delta x = \frac{x_{\max} - x_{\min}}{n} = \frac{10}{n}.
$$
Since $\Delta t$ is proportional to $\Delta x$ (with $\Delta t/\Delta x = 50/10 = 5$), both temporal and spatial errors scale as $O(1/n^2)$.

When plotted on a log-log scale, the error $\epsilon$ as a function of the number of grid points $n$ should follow the relationship:
$$
\epsilon \sim \Delta x^2 \sim \left(\frac{1}{n}\right)^2,
$$
meaning that the error scales quadratically with the grid resolution. In other words, doubling the number of grid points (halving $\Delta x$ and $\Delta t$) should reduce the error by roughly a factor of 4. This quadratic convergence is a hallmark of the second-order accuracy of the Lax-Wendroff scheme.


**Compare to the error made by the explicit upwind scheme. What scheme do you prefer, and why?**


Comparing the error from the explicit upwind scheme to that of the Lax-Wendroff scheme reveals that the upwind method is only first order accurate, so its error decreases linearly with grid refinement. In contrast, the Lax-Wendroff scheme is second order accurate in both space and time, meaning its error decreases quadratically as the grid is refined.

The upwind scheme introduces significant numerical diffusion, causing the amplitude of the solution to decay over time. The Lax-Wendroff method, while it may introduce some dispersive oscillations near steep gradients, better preserves the amplitude and overall shape of the solution for smooth initial conditions.

For these reasons, I prefer the Lax-Wendroff scheme for problems with smooth solutions because its error scales more favorably and it maintains the solution's features with higher accuracy compared to the more diffusive upwind scheme.


### Core task 3

So far, we have seen two finite difference methods. Our third algorithm will use a spectral method.

The central idea is to decompose the solution into Fourier modes of the form
$$
\hat{f} (k,t) = \int_{-\infty}^{+\infty} f (x,t) \,\, \mathrm{e}^{-i k x} dx. \qquad (1)
$$

By applying a Fourier transform to the advection equation, we obtain that the evolution of a Fourier mode is governed by
$$
\frac{\partial \hat{f}}{\partial t} = - i v k \hat{f} , \qquad (2)
$$
which is a simple ODE.

Finally, an inverse Fourier transform allows us to find the solution
$$
f (x,t) = \frac{1}{2 \pi} \int_{-\infty}^{+\infty} \hat{f} (k,t) \,\, \mathrm{e}^{+ i k x} dk . \qquad (3)
$$

Following this idea, we deduce a simple three-step algorithm that allows us to update the numerical solution from ${ f^{n} }$ to ${ f^{n+1} }$:

1. Calculate ${ \hat{f}^{n} }$ by applying a FFT procedure to ${ f^{n} }$.

2. Use equation (2) and your favorite order-1 scheme for ODEs to deduce ${ \hat{f}^{n+1} }$ from ${ \hat{f}^{n} }$.

3. Calculate ${ f^{n+1} }$ by applying a IFFT procedure to $\hat{f}^{n+1}$.

**Write your own spectral scheme**

In [ ]:
def Spectral(v, n_points_space, dx, dt):
    k = np.fft.fftfreq(n_points_space, d=dx) * 2 * np.pi
    def step(f):
        f[:] = np.fft.ifft(np.fft.fft(f) * np.exp(-1j * v * k * dt)).real

    return step

**Sanity-check your code. Under which conditions is it stable? How fast does it converge? What happens when you only use 10 cells in space?**

In [ ]:
# initial condition
X = np.linspace(x_min, x_max, n_points_space, endpoint = True)
f0 = gaussian(X)
f = f0.copy()

# scheme
step = Spectral(v, n_points_space, dx, dt)

# preparation of the animation
fig = plt.figure()
ax = fig.gca()

ax.set_xlim(x_min, x_max)
ax.set_ylim(-0.1, 1.5)

ax.set_xlabel(r'$x$')
ax.set_ylabel(r'$f(x, t)$')

show_f0, = ax.plot(X, f0, 'b--', label = "f0")
show_ft, = ax.plot([], [], 'b', lw = 1, ms = 2, label = "f")
time = ax.annotate(0, xy = (3, 1.4), xytext = (3, 1.4))

# integration
def animate(i):
    step(f)

    show_ft.set_data(X, f)

    time.set_text("t / T = %.2f" % (v * i * dt / (x_max - x_min)))

    return show_f0, show_ft, time

# creation of the animation
anim = animation.FuncAnimation(fig, animate, 10**5, interval = dt * 10, blit = True)

# show the results
plt.show()

<IPython.core.display.Javascript object>

The spectral method for the linear advection equation is **unconditionally stable** when the following conditions are met:

1.   **Periodic Boundary Conditions**: The Fourier transform assumes periodicity. If the domain is not periodic, errors from discontinuities at the boundaries may lead to instability.
2.   **Sufficient Resolution**: While the exponential time-update is exact for each Fourier mode, the method relies on accurately representing the initial condition in Fourier space. If the spatial resolution is too low, aliasing can occur. However, under proper resolution (i.e., when the function is well-resolved on the grid), the method remains stable.


For smooth solutions, spectral methods typically exhibit **exponential convergence**. This means that the error decreases faster than any polynomial rate as the number of grid points increases. In practical terms, if the initial condition is smooth and periodic, increasing the number of cells will reduce the error very rapidly.


When using only 10 cells in space:

1.   **Poor Resolution**: A coarse grid (10 cells) may not capture the features of the solution adequately. The Fourier representation of the initial condition will be crude.
2.   **Aliasing Errors**: With very few points, high-frequency components are misrepresented (or not represented at all), leading to significant aliasing errors.

3.   **Loss of Spectral Accuracy**: Although the method remains stable (provided periodicity is maintained), the convergence benefits are lost. The solution will be highly inaccurate, and the error will be large because the grid is too coarse to resolve the true dynamics.

